In [1]:
# !unzip /home/aistudio/data/data78992/lcqmc.zip -d /home/aistudio/data/
# !unzip /home/aistudio/data/data78992/paws-x-zh.zip -d /home/aistudio/data/
# !unzip /home/aistudio/data/data78992/bq_corpus.zip -d /home/aistudio/data/

## 1. Load dataset

In [2]:
def load_dataset(fpath, num_row_skip=0):

    def read(fp):
        data = open(fp)

        for _ in range(num_row_skip):
            next(data)

        if "test" in fp:
            for line in data:
                line = line.strip().split('\t')
                yield line[0], line[1]
        else:
            for line in data:
                line = line.strip().split('\t')
                if len(line) == 3:
                    yield line[0], line[1], int(line[2])

    if isinstance(fpath, str):
        return list(read(fpath))
    elif isinstance(fpath, (list, tuple)):
        return [list(read(fp)) for fp in fpath]
    else:
        raise TypeError("Input fpath must be a str or a list/tuple of str")

In [3]:
train_set, dev_set, test_set = load_dataset(['./data/paws-x-zh/train.tsv', './data/paws-x-zh/dev.tsv', './data/paws-x-zh/test.tsv'])
# len(train_set), len(dev_set), len(test_set)
train_set = train_set + dev_set

## 2. Transform text

In [4]:
from paddlenlp.datasets import MapDataset
from paddle.io import BatchSampler, DataLoader
from paddlenlp.data import Pad, Stack, Tuple
from paddlenlp.transformers import ErnieGramModel as SeqClfModel
from paddlenlp.transformers import ErnieGramTokenizer as PTMTokenizer
import numpy as np


MODEL_NAME = "ernie-gram-zh"
tokenizer = PTMTokenizer.from_pretrained(MODEL_NAME)


def example_converter(example, max_seq_length, tokenizer):
    text_a, text_b, label = example
    encoded = tokenizer(text=text_a, text_pair=text_b, max_seq_len=max_seq_length)
    input_ids = encoded["input_ids"]
    token_type_ids = encoded["token_type_ids"]
    label = np.array([label], dtype="int64")
    return input_ids, token_type_ids, label


def get_trans_fn(max_seq_length=128, tokenizer=tokenizer):
    return lambda ex: example_converter(ex, max_seq_length, tokenizer)


batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id), 
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),
    Stack(dtype="int64")
    ): fn(samples)


def create_dataloader(dataset, 
                      trans_fn, 
                      batchify_fn, 
                      test=False,
                      batch_size=128, 
                      shuffle=True, 
                      sampler=BatchSampler):
    
    if test:
        dataset = [d + (0,) for d in dataset]

    if not isinstance(dataset, MapDataset):
        dataset = MapDataset(dataset)
        
    dataset.map(trans_fn)
    batch_sampler = sampler(dataset, 
                            shuffle=shuffle, 
                            batch_size=batch_size)
    
    dataloder = DataLoader(dataset, 
                           batch_sampler=batch_sampler, 
                           collate_fn=batchify_fn)
    
    return dataloder

[2022-03-25 09:29:46,050] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-gram-zh/vocab.txt


In [5]:
max_seq_length = 128; batch_size = 64
trans_fn = get_trans_fn(max_seq_length)
train_loader = create_dataloader(train_set, trans_fn, batchify_fn, batch_size=batch_size)
# dev_loader = create_dataloader(dev_set, trans_fn, batchify_fn, batch_size=batch_size)
test_loader = create_dataloader(test_set, trans_fn, batchify_fn, shuffle=False, test=True, batch_size=batch_size)

## 3. Model building

In [13]:
from paddle import nn
import paddle


class PTM(nn.Layer):

    def __init__(self, pretrained_model, dropout=0.1, num_class=2):
        super().__init__()

        self.ptm = pretrained_model
        ptm_out_dim = self.ptm.config["hidden_size"]
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(ptm_out_dim, ptm_out_dim // 2)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(ptm_out_dim // 2, num_class)

    def encoder(self, input_ids, token_type_ids):
        _, embd = self.ptm(input_ids, token_type_ids)
        embd = self.dropout(embd)
        return embd

    def forward(self, input_ids, token_type_ids):
        embd = self.encoder(input_ids, token_type_ids)
        hidden = self.relu(self.fc1(embd))
        logits = self.fc2(hidden)
        return logits

In [16]:
from paddlenlp.transformers import LinearDecayWithWarmup

epoch = 10
weight_decay = 0.01
warmup_proportion = 0.1
lr_scheduler = LinearDecayWithWarmup(4e-5, len(train_loader) * epoch,
                                         warmup_proportion)

def get_model(model):
    decay_params = [
        p.name for n, p in model.named_parameters()
        if not any(nd in n for nd in ["bias", "norm"])
    ]
    optimizer = paddle.optimizer.AdamW(
    parameters=model.parameters(), 
    learning_rate=lr_scheduler, 
    weight_decay=weight_decay, 
    apply_decay_param_fun=lambda x: x in decay_params)

    criterion = paddle.nn.CrossEntropyLoss()

    model = paddle.Model(model)
    metric = paddle.metric.Accuracy()
    model.prepare(optimizer, criterion, metric)
    return model

In [17]:
ptm = SeqClfModel.from_pretrained(MODEL_NAME)
model = PTM(ptm)
model = get_model(model)

[2022-03-25 09:34:59,952] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-gram-zh/ernie_gram_zh.pdparams


## 4. Model training

In [18]:
model.fit(train_loader, epochs=epoch, verbose=2, log_freq=100)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 100/799 - loss: 0.6856 - acc: 0.4938 - 404ms/step
step 200/799 - loss: 0.6622 - acc: 0.5162 - 405ms/step
step 300/799 - loss: 0.6472 - acc: 0.5334 - 406ms/step
step 400/799 - loss: 0.5190 - acc: 0.5644 - 406ms/step
step 500/799 - loss: 0.4061 - acc: 0.6054 - 406ms/step
step 600/799 - loss: 0.3992 - acc: 0.6405 - 406ms/step
step 700/799 - loss: 0.3925 - acc: 0.6696 - 407ms/step
step 799/799 - loss: 0.1941 - acc: 0.6923 - 407ms/step
Epoch 2/10
step 100/799 - loss: 0.3036 - acc: 0.8717 - 409ms/step
step 200/799 - loss: 0.2707 - acc: 0.8784 - 408ms/step
step 300/799 - loss: 0.2742 - acc: 0.8791 - 409ms/step
step 400/799 - loss: 0.3198 - acc: 0.8807 - 409ms/step
step 500/799 - loss: 0.2394 - acc: 0.8832 - 409ms/step
step 600/799 - loss: 0.2263 - acc: 0.8847 - 409ms/step
step 700/799 - loss: 0.1447 - acc: 0.8859 - 409ms/step
step 799/799 - loss: 0.2232 - acc: 0.8876 -

## 5. Prediction

In [19]:
import paddle.nn.functional as F


predictions = []
logits = model.predict(test_loader)

for batch in logits[0]:
    batch = paddle.to_tensor(batch)
    probs = F.softmax(batch, axis=1)
    preds = paddle.argmax(probs, axis=1).numpy().tolist()
    predictions.extend(preds)

Predict begin...
step 32/32 [==============================] - ETA: 7s - 234ms/st - ETA: 5s - 202ms/st - ETA: 4s - 192ms/st - ETA: 4s - 187ms/st - ETA: 4s - 184ms/st - ETA: 3s - 181ms/st - ETA: 3s - 179ms/st - ETA: 2s - 178ms/st - ETA: 2s - 179ms/st - ETA: 2s - 177ms/st - ETA: 1s - 177ms/st - ETA: 1s - 176ms/st - ETA: 1s - 176ms/st - ETA: 0s - 175ms/st - ETA: 0s - 172ms/st - 167ms/step          
Predict samples: 2000


In [20]:
with open('paws-x.tsv', 'w') as f:
    f.write("index\tprediction")
    for idx, p in enumerate(predictions):
        f.write(f"\n{idx}\t{p}")
    f.close()